# Ingest student data
https://docs.wonde.com/docs/api/sync#students

includes education_details

In [53]:
%run _Config

StatementMeta(, 008943c9-8333-47c6-a86a-1b2253171c96, -1, Finished, Available)

In [54]:
%run _WondeStructs

StatementMeta(, 008943c9-8333-47c6-a86a-1b2253171c96, -1, Finished, Available)

In [55]:
%run _WondeAPI

StatementMeta(, 008943c9-8333-47c6-a86a-1b2253171c96, -1, Finished, Available)

### Get list of available schools

In [56]:
dfSchoolIds = spark.read.load(BronzeBasePath+"/Schools")
display(dfSchoolIds)

StatementMeta(, 008943c9-8333-47c6-a86a-1b2253171c96, 77, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9318af47-2b68-4eeb-a2f6-f1760aef1d89)

### Iterate through schools and get students

In [60]:
def get_allstudents():
    for school in dfSchoolIds.collect():
        url = baseWondeUrl + "schools/"+school["id"]+"/students?include=education_details"
        df = getWondeAPI(url, wondeHeaders, structWondeStudent)
        df = df.withColumn("school_id", lit(school["id"]))
        yield df

StatementMeta(, 008943c9-8333-47c6-a86a-1b2253171c96, 81, Finished, Available)

In [61]:
for school in get_allstudents():
    display(school)

StatementMeta(, 008943c9-8333-47c6-a86a-1b2253171c96, 82, Finished, Available)

SynapseWidget(Synapse.DataFrame, d5fbff15-e1cd-423c-9aad-883fd8f6d6db)

### Write to data lake

In [65]:
# Purge existing data
# Is this "best practice" or should bronze layer literally collect everything?
# Probably more efficient to build something using query parameter &updated_after= and delta merge
try:
    mssparkutils.fs.rm(BronzeBasePath + "/Students", True)
except:
    print("Path doesn't exist, most likely already deleted.")

# Write new school data into delta
for school in get_allstudents():
    dfFinal = (school.select(
                col("*"),
                col("education_details.data.upn").alias("upn"),
                col("updated_at.date").alias("updated_at_date"))
                .drop("edcuation_details")
                .drop("updated_at") 
            )
    (dfFinal.write
        .mode("append")
        .format("delta")
        .save(BronzeBasePath + "/Students")
    )


StatementMeta(, 008943c9-8333-47c6-a86a-1b2253171c96, 86, Finished, Available)